# Aufgabe 05
Black Jack

Der Type Card besteht aus einer Karte und einer Liste aus Karten. Welche Karte dann bei einer neuen Hand als erste Karte genommen wird ist noch offen.
Der Rank wird als enum definiert und dann mit case .. of werte zugewiesen. Die Alternative wäre gewesen, den Rank als `deriving(Enum)` zu definieren, dann hat jeder von den Attributen einen Int als Wert. Allerdings sollen J,Q,K den gleichen Wert haben

In [70]:
data Rank = R2 | R3 | R4 | R5 | R6 | R7 | R8 | R9 | R10 | Jack | Queen | King | Ace deriving (Show, Eq, Bounded, Enum)
data Suit = Hearts | Diamonds | Spades | Clubs deriving (Show, Eq, Bounded, Enum)
data Card = Card Suit Rank deriving (Show, Eq, Bounded)
data Hand = Empty | Hand Card [Card] deriving (Show, Eq)

rank :: Rank -> Int
rank x = case x of 
    R2 -> 2
    R3 -> 3
    R4 -> 4
    R5 -> 5
    R6 -> 6
    R7 -> 7
    R8 -> 8
    R9 -> 9
    R10 -> 10
    Jack -> 10
    Queen -> 10
    King -> 10
    Ace -> 11



## Functions

### <+>
- Die Funktion bekommt 2 Hände, in jeder hand ist eine karte und eine liste von Karten.
- es wird die *Hauptkarte* von der ersten Hand auch als *Hauptkarte* von der zurückgegebenen Hand genutzt
- Zusätzlich wird die *Hauptkarte* der zweiten Hand zu den anderen beiden Karten-Listen konkatiniert, nachdem sie in eine Liste konvertiert wird

In [71]:
getCardValue :: Card -> Int
getCardValue (Card _ n)  = rank n

getCards :: Hand -> [Card]
getCards (Hand _ x) = x

getCard :: Hand -> Card
getCard (Hand x _) = x

(<+>) :: Hand -> Hand -> Hand
(<+>) Empty h = h
(<+>) h Empty = h
(<+>) Empty Empty = Empty
(<+>) (Hand c1 cs1) (Hand c2 cs2) = Hand c1 ([c2] ++ cs1 ++ cs2)


In [72]:
let card = Card Clubs R3
let card2 = Card Spades R2
let card3 = Card Spades Ace
let card5 = Card Clubs Ace
let card6 = Card Diamonds Ace
let card7 = Card Hearts Ace
let hand = Hand card [card2, card3, card5, card6, card7]

let card4 = Card Diamonds R10
print(hand <+> Hand card3 [card, card4])

Hand (Card Clubs R3) [Card Spades Ace,Card Spades R2,Card Spades Ace,Card Clubs Ace,Card Diamonds Ace,Card Hearts Ace,Card Clubs R3,Card Diamonds R10]


### FullDeck
- ziemlich faszinierende Sache: fullDeck erstellt eine Liste mit einer List-comprehension, unzwar aus den einzelnen Listen Suit und Rank. Man kann aus Card nicht direkt eine komplette Bound liste machen, da es nicht teil von Enum ist. Und kann es auch nicht sein, weil die Werte wieder mehrere Werte haben.
- Also gehen wir durch alle werte durch und erstellen für jeden dieser Werte eine Hand mit den Attributen
- um eine Hand zu erstellen, brauchen wir eine Hauptkarte und den Rest der Karten als array
- also wird noch die erste Karte und dann der rest als array zurückgegeben
### numOfAces
- auch wieder list comprehension: erstelle eine Liste von Karten, wobei nur die karten zur liste hinzugefügt werden, welche den spezifischen Rang haben
### getValue
- diesmal etwas komplizierter
- man muss tracken, wie viele asse schon 'verwendet' wurden
- daher eine helper funktion, die nichts anderes macht als sich selbst immer wieder aufzurufen, unzwar so lange wie asse da sind, bis der wert unter oder gleich 21 ist. Dazu wird der wert immer -10 und die Asse -1 gerechnet, bis entweder keine asse mehr da sind oder 21 erreicht ist

In [73]:
fullDeck :: Hand
fullDeck = Hand (head deck) (tail deck) where
    deck = [Card suit rank | suit <- [minBound .. maxBound] :: [Suit], rank <- [minBound .. maxBound] :: [Rank]]

getCardsByRank :: Hand -> Int -> [Card]
getCardsByRank (Hand c cs) n = [Card s r | Card s r <- c:cs, rank r == n]

numOfAces :: Hand -> Int
numOfAces h = length (getCardsByRank h 11)

getValue :: Hand -> Int
getValue (Hand c cs) = getValue' (sum [rank r | Card s r <- c:cs]) (numOfAces (Hand c cs)) where
    getValue' :: Int -> Int -> Int
    getValue' s a
        | s > 21 && a > 0 = getValue' (s - 10) (a - 1)
        | otherwise = s


In [74]:
deck = fullDeck
-- print(getCardsByRank deck 11)
-- print (numOfAces deck)
print(getValue hand)

19